# Import

In [1]:
import os
import matplotlib.pyplot as plt
plt.close('all')
import dfm_tools as dfmt
import hydrolib.core.dflowfm as hcdfm 
import xarray as xr
import pandas as pd
import numpy as np
import contextily as ctx

import solara
import solara.lab
from ipyleaflet import Map, basemaps
import datetime
from pathlib import Path


# Initial set-ups, definitions, etc.

In [2]:
m = solara.reactive(Map(center=(18.125, 106.325), zoom=10, scroll_wheel_zoom=True, 
                        basemap=basemaps.OpenStreetMap.Mapnik))
def reset_map():
    m.set(Map(center=(lat_min.value+(lat_max.value-lat_min.value)*0.5, lon_min.value+(lon_max.value-lon_min.value)*0.5), 
              zoom=10, scroll_wheel_zoom=True, basemap=basemaps.OpenStreetMap.Mapnik))


# User Input

NEW:

In [4]:
####################################### User Input ########################################
model_name = solara.reactive("Model Name")
continuous_update = solara.reactive(True)

model_resolution_options = ["0.05", "0.5"]
dxy = solara.reactive("0.5")

lat_min = solara.reactive(0.0); lat_max = solara.reactive(0.0); lon_min = solara.reactive(0.0); lon_max = solara.reactive(0.0)

date_min = solara.reactive(datetime.date(2022, 11, 1))
date_max = solara.reactive(datetime.date(2022, 11, 3))
ref_date = solara.reactive(datetime.date(2022, 1, 1))

dir_output = solara.reactive("C:\\") # os.path.abspath(f'./{model_name}_model')
def makedir():
    dir_path = Path(dir_output.value)  
    os.makedirs(dir_path, exist_ok=True)
    dir_output_data = dir_path / 'data'
    os.makedirs(dir_output_data, exist_ok=True)
overwrite = False # used for downloading of forcing data. Always set to True when changing the domain
crs = 'EPSG:4326' # coordinate reference system

# ----------------------------------------------------------------------------------------------------


@solara.component
def Tab_User_Input():
    solara.use_effect(reset_map, dependencies=[model_name.value])
    with solara.Card("User Input", style={"width": "100%", "padding": "10px"}):
        solara.Markdown("""**Note to the user**: In this notebook we use publicly available data 
                        from Copernicus Programme of the European Union. To access this data you 
                        need to create accounts at 
                        [Copernicus Marine Service](https://data.marine.copernicus.eu/register)
                        and the [Climate Data Store](https://cds.climate.copernicus.eu/profile).
                        Do not forget to accept the CDS license agreement.""")
        solara.InputText("Model Name", value=model_name, continuous_update=continuous_update.value)
        solara.InputText("Output directory", value=dir_output, continuous_update=True) # --------- Only works for now as input text, file explorer not successful yet
        solara.Button(label="Select/Create directory",on_click=makedir)

        solara.Markdown("Area of Model & Resolution:")
        solara.InputFloat("Lat minimum", value=lat_min, continuous_update=continuous_update.value)
        solara.InputFloat("Lat maximum", value=lat_max, continuous_update=continuous_update.value)
        solara.InputFloat("Lon minimum", value=lon_min, continuous_update=continuous_update.value)
        solara.InputFloat("Lon maximum", value=lon_max, continuous_update=continuous_update.value)
        solara.Select(label="Model Resolution", value=dxy, values=model_resolution_options)

        solara.Markdown("Date selection:")
        solara.Text("Select min date:"); solara.lab.InputDate(date_min)
        solara.Text("Select max date:"); solara.lab.InputDate(date_max)
        solara.Text("Select reference date:"); solara.lab.InputDate(ref_date)
        if date_max.value < date_min.value:
            solara.Markdown("**Warning**: The end date cannot be earlier than the start date.", 
                            style={"color": "red"})
            
Tab_User_Input()



Cannot show ipywidgets in text

In [ ]:
# Outputs I might need later:
# model_name.value # --> model name
# dir_output.value # --> path for output; looks different when doing print() --> MIGHT NEED SPECIAL ATTENTION/HANDLING LATER!!!!
# lat_min.value # --> lat_min; same with lat_max and lon_min, lon_max
# float(dxy.value) # --> resolution as number
# date_min.value.strftime("%Y-%m-%d") # --> date min in approriate format
# date_max.value.strftime("%Y-%m-%d") # --> date max in approriate format
# ref_date.value.strftime("%Y-%m-%d") # --> ref date in approriate format

# Grid Generation and Refinement

In [ ]:
#continue here

# generate spherical regular grid
mk_object = dfmt.make_basegrid(lon_min.value, lon_max.value, lat_min.value, lat_max.value, dx=dxy, dy=dxy, crs=crs)


# generate plifile from grid extent and coastlines
bnd_gdf = dfmt.generate_bndpli_cutland(mk=mk_object, res='h', buffer=0.01)
bnd_gdf_interp = dfmt.interpolate_bndpli(bnd_gdf, res=0.03)
pli_polyfile = dfmt.geodataframe_to_PolyFile(bnd_gdf_interp, name=f'{model_name.value}_bnd')
poly_file = os.path.join(dir_output, f'{model_name}.pli')
pli_polyfile.save(poly_file)

# Putting everything together

put code of single tabs in own code cell, per tab --> to have it more structured!

In [ ]:



###########################################################################################

@solara.component
def Tab2():
    
    with solara.Card("xxx2", style={"width": "100%", "padding": "10px"}):
        solara.Text("text")

@solara.component
def Tab3():
    
    with solara.Card("xxx3", style={"width": "100%", "padding": "10px"}):
        solara.Text("text")


@solara.component
def Tabxxx():
    
    with solara.Card("xxx", style={"width": "100%", "padding": "10px"}):
        solara.Text("text")

###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
###########################################################################################
selected_tab = solara.reactive('UserInput') 
@solara.component
def SettingsTabs():
    with solara.Column(style={"width": "100%", "align-items": "center"}):
        with solara.Row(gap="10px", style={"justify-content": "flex-start", "width": "80%"}):
            solara.Button("User Input", on_click=lambda: selected_tab.set('UserInput'))
            solara.Button("2", on_click=lambda: selected_tab.set('2'))
            solara.Button("3", on_click=lambda: selected_tab.set('3'))
        with solara.Row(gap="10px", style={"justify-content": "flex-start", "width": "80%"}):
            solara.Button("xxx", on_click=lambda: selected_tab.set('xxx'))
 
    if selected_tab.value == 'UserInput':
        Tab_User_Input()
    elif selected_tab.value == '2':
        Tab2()
    elif selected_tab.value == '3':
        Tab3()
    elif selected_tab.value == 'xxx':
        Tabxxx()

@solara.component
def Page():
    with solara.Columns():
        with solara.Column(style={"width": "70%", "min-width": "650px"}):
            display(m.value)

        with solara.Column(style={"width": "30%", "min-width": "500px"}):
            SettingsTabs()

Page()